In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [4]:
train = pd.read_csv("D:/Analytical Problem/Loan Prediction problem/train.csv")
test = pd.read_csv("D:/Analytical Problem/Loan Prediction problem/test.csv")

In [5]:
train.head(3)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y


In [6]:
ntr = len(train)
all_data = train.append(test,ignore_index=True,sort=True)

In [7]:
all_data.shape

(981, 13)

In [8]:
all_data.isnull().sum()

ApplicantIncome        0
CoapplicantIncome      0
Credit_History        79
Dependents            25
Education              0
Gender                24
LoanAmount            27
Loan_Amount_Term      20
Loan_ID                0
Loan_Status          367
Married                3
Property_Area          0
Self_Employed         55
dtype: int64

In [9]:
all_data[all_data.Credit_History.isnull()].groupby("Loan_Status").count()

,ApplicantIncome,CoapplicantIncome,Credit_History,Dependents,Education,Gender,LoanAmount,Loan_Amount_Term,Loan_ID,Married,Property_Area,Self_Employed
Loan_Status,,,,,,,,,,,,
N,13,13,0,13,13,13,12,13,13,13,13,8
Y,37,37,0,37,37,36,37,37,37,37,37,36


In [10]:
all_data.Credit_History.fillna(all_data.Credit_History.mode()[0], inplace=True)
all_data.Dependents.fillna(all_data.Dependents.mode()[0],inplace=True)
all_data.Gender.fillna(all_data.Gender.mode()[0],inplace=True)
all_data.Loan_Amount_Term.fillna(all_data.Loan_Amount_Term.mode()[0],inplace=True)
all_data.Married.fillna(all_data.Married.mode()[0],inplace=True)
all_data.Self_Employed.fillna(all_data.Self_Employed.mode()[0],inplace=True)

In [11]:
pvt = all_data.pivot_table(index="Self_Employed",columns="Education",values="LoanAmount",aggfunc = np.median)
def lnamt(x):
    return pvt.loc[x['Self_Employed'],x['Education']]
all_data.LoanAmount.fillna(all_data[all_data.LoanAmount.isnull()].apply(lnamt,axis=1),inplace=True)

In [12]:
all_data.isnull().sum()

ApplicantIncome        0
CoapplicantIncome      0
Credit_History         0
Dependents             0
Education              0
Gender                 0
LoanAmount             0
Loan_Amount_Term       0
Loan_ID                0
Loan_Status          367
Married                0
Property_Area          0
Self_Employed          0
dtype: int64

In [13]:
all_data.drop(["Loan_ID"],axis=1,inplace=True)
cat_var = list(all_data.select_dtypes(["object"]).columns.values)
print cat_var

['Dependents', 'Education', 'Gender', 'Loan_Status', 'Married', 'Property_Area', 'Self_Employed']


In [14]:
all_data.select_dtypes(["object"]).head(2)

,Dependents,Education,Gender,Loan_Status,Married,Property_Area,Self_Employed
0,0,Graduate,Male,Y,No,Urban,No
1,1,Graduate,Male,N,Yes,Rural,No


In [15]:
from sklearn.preprocessing import LabelEncoder

In [17]:
tf_var = [x for x in cat_var if x != "Loan_Status"]
print tf_var

['Dependents', 'Education', 'Gender', 'Married', 'Property_Area', 'Self_Employed']


In [18]:
le = LabelEncoder()
for v in tf_var:
    all_data[v] = le.fit_transform(all_data[v])

In [19]:
all_data[cat_var].head(3)

,Dependents,Education,Gender,Loan_Status,Married,Property_Area,Self_Employed
0,0,0,1,Y,0,2,0
1,1,0,1,N,1,0,0
2,0,0,1,Y,1,2,1


In [20]:
all_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 12 columns):
ApplicantIncome      981 non-null int64
CoapplicantIncome    981 non-null float64
Credit_History       981 non-null float64
Dependents           981 non-null int64
Education            981 non-null int64
Gender               981 non-null int64
LoanAmount           981 non-null float64
Loan_Amount_Term     981 non-null float64
Loan_Status          614 non-null object
Married              981 non-null int64
Property_Area        981 non-null int64
Self_Employed        981 non-null int64
dtypes: float64(4), int64(7), object(1)
memory usage: 92.0+ KB


In [21]:
all_data['TotalIncome'] = all_data.CoapplicantIncome + all_data.ApplicantIncome
all_data['TotalIncome_log'] = np.log(all_data['TotalIncome'])
all_data['LoanAmount_log'] = np.log(all_data['LoanAmount'])
to_rm = ['CoapplicantIncome','ApplicantIncome','TotalIncome','LoanAmount']

In [35]:
df = all_data.copy()

In [36]:
df.drop(to_rm, axis=1, inplace= True)

In [37]:
df.Loan_Amount_Term.value_counts().sort_values()

350.0      1
6.0        1
12.0       2
36.0       3
60.0       3
120.0      4
84.0       7
240.0      8
300.0     20
480.0     23
180.0     66
360.0    843
Name: Loan_Amount_Term, dtype: int64

In [38]:
train_data = df[:ntr].copy()
test_data = df[ntr:].copy()

In [41]:
train_data.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 11 columns):
Credit_History      614 non-null float64
Dependents          614 non-null int64
Education           614 non-null int64
Gender              614 non-null int64
Loan_Amount_Term    614 non-null float64
Loan_Status         614 non-null object
Married             614 non-null int64
Property_Area       614 non-null int64
Self_Employed       614 non-null int64
TotalIncome_log     614 non-null float64
LoanAmount_log      614 non-null float64
dtypes: float64(4), int64(6), object(1)
memory usage: 52.8+ KB


In [42]:
test_data.drop("Loan_Status",axis=1,inplace=True)

In [45]:
train_data.isnull().sum()

Credit_History      0
Dependents          0
Education           0
Gender              0
Loan_Amount_Term    0
Loan_Status         0
Married             0
Property_Area       0
Self_Employed       0
TotalIncome_log     0
LoanAmount_log      0
dtype: int64

In [48]:
train_data.Loan_Status.replace({'Y':1,'N':0},inplace = True)

In [49]:
train_data.Loan_Status.value_counts()

1    422
0    192
Name: Loan_Status, dtype: int64